# Azure AI Agent service - Many agents

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
import os
import sys

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
from openai import AzureOpenAI

In [2]:
load_dotenv("azure.env")

True

In [3]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

## Project

In [4]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.getenv("PROJECT_CONNECTION_STRING"),
)

In [5]:
model = "gpt-4o"

In [6]:
agent1 = project_client.agents.create_agent(
    model=model,
    name="Q&A",
    instructions=("You are an AI agent."),
)

print(f"Created Q&A agent, agent ID: {agent1.id}")

Created Q&A agent, agent ID: asst_NAyoEhFV2ZOa3jwjmXtyzd4r


In [7]:
agent2 = project_client.agents.create_agent(
    model=model,
    name="Translator",
    instructions=("You are an AI agent that translate english into french")
)

print(f"Created Translator agent, agent ID: {agent2.id}")

Created Translator agent, agent ID: asst_8izGaUiVcVMGv8eVpVk6BcEu


In [8]:
agent3 = project_client.agents.create_agent(
    model=model,
    name="Emojis",
    instructions=("You are an AI agent that add emojis to a text")
)

print(f"Created Emojis agent, agent ID: {agent3.id}")

Created Emojis agent, agent ID: asst_64ACPe1rDi805LYQlLlsZ5kb


In [9]:
thread = project_client.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

Created thread, thread ID: thread_HL6L34Xn7sBbOAKPR5lEZWyC


In [10]:
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=("What is a GenAI model?"),
)

print(f"Message created, message ID: {message.id}")

Message created, message ID: msg_cCrqcfTzlPGFU7SLnln9PwTz


In [11]:
run = project_client.agents.create_and_process_run(thread_id=thread.id,
                                                   agent_id=agent1.id)

print(f"Run finished for Agent1 with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished for Agent1 with status: completed


In [12]:
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Thread messages: {messages}")

# Print last message
last_msg = messages.get_last_text_message_by_role("assistant")

if last_msg:
    print(f"\nAgent1 last message:\n{last_msg.text.value}")

Thread messages: {'object': 'list', 'data': [{'id': 'msg_qbixsn94mKk3BjeSgd07kjjR', 'object': 'thread.message', 'created_at': 1747746156, 'assistant_id': 'asst_NAyoEhFV2ZOa3jwjmXtyzd4r', 'thread_id': 'thread_HL6L34Xn7sBbOAKPR5lEZWyC', 'run_id': 'run_4TnjZc4O0JNlgvpzTLfOJHOG', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': "A **Generative AI (GenAI) model** refers to a type of artificial intelligence model designed to generate new, original content, such as text, images, audio, code, or other forms of data, based on the patterns found in the training data it was exposed to. It leverages deep learning algorithms, often involving neural networks like transformers or GANs (Generative Adversarial Networks), to predict or generate outputs that mimic human creation.\n\n### Characteristics of GenAI Models:\n1. **Creation of Content**: GenAI models can produce natural language text (e.g., essays, responses, stories), images, videos, or other types of generative outputs.\n2.

In [13]:
run = project_client.agents.create_and_process_run(thread_id=thread.id,
                                                   agent_id=agent2.id)

print(f"Run finished for Agent2 with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished for Agent2 with status: completed


In [14]:
messages = project_client.agents.list_messages(thread_id=thread.id)

# Print last message
last_msg = messages.get_last_text_message_by_role("assistant")

if last_msg:
    print(f"Agent last message:\n{last_msg.text.value}")

Agent last message:
Un modèle **IA générative (GenAI)** désigne un type de modèle d'intelligence artificielle conçu pour créer du contenu original, comme du texte, des images, des sons, du code ou d'autres formes de données, en se basant sur les motifs présents dans les données d'entraînement auxquelles il a été exposé. Il utilise des algorithmes d'apprentissage approfondi, souvent avec des réseaux neuronaux tels que les transformateurs ou les GANs (Réseaux antagonistes génératifs), pour prédire ou générer des contenus qui imitent les créations humaines.

### Caractéristiques des modèles GenAI :
1. **Création de contenu** : Les modèles GenAI peuvent produire du texte (ex. : des essais, réponses, récits), des images, des vidéos ou d'autres types de résultats génératifs.
2. **Pré-entraînement sur de larges ensembles de données** : Ces modèles sont entraînés sur des ensembles de données considérables (par exemple, textes sur Internet pour les modèles basés sur le langage ou images pour la

In [15]:
run = project_client.agents.create_and_process_run(thread_id=thread.id,
                                                   agent_id=agent3.id)

print(f"Run finished for Agent3 with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

Run finished for Agent3 with status: completed


In [16]:
messages = project_client.agents.list_messages(thread_id=thread.id)

# Print last message
last_msg = messages.get_last_text_message_by_role("assistant")

if last_msg:
    print(f"Agent last message:\n{last_msg.text.value}")

Agent last message:
A **GenAI model** 🤖✨ is a type of artificial intelligence designed to **generate original content** like text 📝, images 🖼️, audio 🎵, videos 🎥, or even code 💻. It learns patterns from large datasets and produces human-like results using technologies such as **deep learning** and neural networks like **transformers** or **GANs**.

### 🌟 Key Features of GenAI Models:
1. **Creative Generation**: Produces responses, art, images, or ideas based on input. 🎨✍️
2. **Contextual Understanding**: Recognizes and uses context to generate high-quality, coherent content. 🤔✨
3. **Large-Scale Training**: Trained on massive datasets (e.g., text from the web, image libraries). 📚💾
4. **Multi-Modal Capabilities**: Some can handle multiple forms of input and output, like text-to-image generation. 🔄

### ⚡ Examples of GenAI Models:
- **Text Generation**: ChatGPT, GPT-4, or Google Bard for conversational text. 🗨️
- **Image Creation**: DALL·E 2, Stable Diffusion, or MidJourney. 🖌️
- **Audio 

## Post processing

In [17]:
agents = project_client.agents.list_agents()

for i in range(len(agents.data)):
    print(agents.data[i])
    print()

{'id': 'asst_UzWDa2x48dwZN2nNql1SK7xw', 'object': 'assistant', 'created_at': 1747746214, 'name': 'document_agent', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are an AI helpful agent to analyse document', 'tools': [{'type': 'file_search'}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {'file_search': {'vector_store_ids': ['vs_fdQ9fF7pJ5C4ZaAZGxH0jzMt']}}, 'metadata': {}, 'response_format': 'auto'}

{'id': 'asst_64ACPe1rDi805LYQlLlsZ5kb', 'object': 'assistant', 'created_at': 1747746145, 'name': 'Emojis', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are an AI agent that add emojis to a text', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}

{'id': 'asst_8izGaUiVcVMGv8eVpVk6BcEu', 'object': 'assistant', 'created_at': 1747746145, 'name': 'Translator', 'description': None, 'model': 'gpt-4o', 'instructions': 'You are an AI agent that translate english into french', 'tools': [], 'top_p': 1.

In [18]:
project_client.agents.delete_agent(agent1.id)

{'id': 'asst_NAyoEhFV2ZOa3jwjmXtyzd4r', 'object': 'assistant.deleted', 'deleted': True}

In [19]:
project_client.agents.delete_agent(agent2.id)

{'id': 'asst_8izGaUiVcVMGv8eVpVk6BcEu', 'object': 'assistant.deleted', 'deleted': True}

In [20]:
project_client.agents.delete_agent(agent3.id)

{'id': 'asst_64ACPe1rDi805LYQlLlsZ5kb', 'object': 'assistant.deleted', 'deleted': True}